In [2]:
import openai, os, chromadb, json

openai.api_key = os.environ.get('OPENAI_API_KEY')

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


NameError: name 'client' is not defined

In [5]:
def query(prompt, **kwargs):
    """
    wrapper for the API to save the prompt and the result
    """
    args = {
    "engine":"text-davinci-003",
    "temperature":0,
    "max_tokens":1000,
    "stop":"END",
    }

    for key in kwargs:
        args[key] = kwargs[key]

    r = openai.Completion.create(prompt=prompt, **args)
    return r["choices"][0]["text"].strip()

In [6]:
theme = "dystopian universe"

In [7]:
generatePlotPrompt = """ /* Generate an Plot for a movie

type character{{
  "name": str,
  "background": str
}}

interface overview {{
  "setting": str, //where the section takes place
  "characters": character[], //the character names
  "conflict": str, //what conflict needs to get solved
  "plotTwist": str, //any plot twists to occur
  "resolution": str // how this plot concludes
}}

const generatePlot = (theme) : outline => {{
    return ai.generatePlot(theme)
}}


theme = '{}'
//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generatePlot(theme))
Out:"""

In [8]:
generateScenesPrompt = """ /* Generate scenes for a movie

type scene{{
  "name": str, // the name of the scene
  "summary": str, // a detailed description with specifics of what happens in this scene
}}

const generateScenes = (plot) : scene[] =>{{
    return ai.generateScenes(plot)
}}

plot = '{}'

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generatePlot(plot))
Out:"""

In [9]:
generateDialoguesPrompt = """ /* Write the next set of dialogues in the given scene of the movie

//the overview of the entire movie
movieOverview = {}

//the outline that the movie should follow
movieOutline = {}

//the paragraphs of the movie so far
dialoguesSoFar = {}

interface scene {{
  "dialogues": dialogue[] //all of the dialogues in the story
}}

type dialogue{{
  "name": str, // the name of the speaker
  "text": str, // dialogue that speaker says
}}

//generate multiple longform dialogues for this scene and list them
const {} = (currentScene) : dialogues[] =>{{
    return ai.write()
}}

currentSceneDescription = {}

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + {}(currentOutlineSection))
Out:"""

In [10]:
generateDialoguesPrompt = """ /* Write the next set of dialogues in the movie for the given scene

//the Plot of the entire movie
plot = {}

//the outline that the movie should follow
allScenes = {}

//list of dialogues that have already been spoken
dialoguesSoFar = {}

type dialogue{{
  "name": str, // the name of the speaker
  "text": str, // dialogue that speaker says
}}

//generate multiple longform dialogues for this scene and list them
const {} = (currentScene) : dialogue[] =>{{
    return ai.write()
}}

currentSceneDescription = {}

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + {}(currentOutlineSection))
Out:"""

In [24]:
from uuid import uuid4
uuid4().hex

'b57c5f5ddfb345769751f6a5a2fa8b32'

In [32]:
rawPlot = query(generatePlotPrompt.format(theme), temperature=0.5)
parsedPlot = json.loads(rawPlot)
print('Overall Plot')
display(parsedPlot)

rawScenes = query(generateScenesPrompt.format(rawPlot), temperature=.5)
parsedScenes = json.loads(rawScenes)
print('Overall Scenes')
display(parsedScenes)

dialoguesSoFar = []
chroma_client = chromadb.Client()
try:
    chroma_client.delete_collection(name="my_collection")
except:
    pass
collection = chroma_client.create_collection(name="my_collection")
for scenedidx, scene in enumerate(parsedScenes):

    subTheme = f"{scene['name']}: {scene['summary']}\n"
    print(subTheme)

    rawSubPlot = query(generatePlotPrompt.format(subTheme), temperature=0.5)
    parsedSubPlot = json.loads(rawSubPlot)
    print('Sub Plot')
    display(parsedSubPlot)
    
    rawSubScenes = query(generateScenesPrompt.format(rawSubPlot), temperature=.5)
    parsedSubScenes = json.loads(rawSubScenes)
    print('Sub Scenes')
    display(parsedSubScenes)

    for subScene in parsedSubScenes:
        try:
            similarDialogues = collection.query(
                query_texts=[subScene['summary']],
                n_results=2
            )['documents']
            print('similarDialogues', similarDialogues)
        except:
            similarDialogues = []
        
        try:
            print('passed dialouges', similarDialogues + dialoguesSoFar[-10:])
            rawDialogues = query(generateDialoguesPrompt.format(rawSubPlot, rawSubScenes, similarDialogues + dialoguesSoFar[-10:], subScene['name'], subScene['summary'], subScene['name']))
            print(f'Dialogues generated: {len(json.loads(rawDialogues))}')
            parsedDialoges = json.loads(rawDialogues)
            print('parsed good')
            recordedDialoges = []
            for dialogue in parsedDialoges:
                dialogue['id'] = str(uuid4().hex)
                collection.add(documents=[json.dumps({'name':dialogue['name'], 'text':dialogue['text']})], metadatas=[{"source": "my_source"}], ids=[dialogue['id']])
                recordedDialoges.append(dialogue)
            
            dialoguesSoFar += recordedDialoges
        except Exception as e:
            try:
                print('FAILED, TRYING AGAIN', e)
                rawDialogues = query(generateDialoguesPrompt.format(rawSubPlot, rawSubScenes, dialoguesSoFar[-10:], subScene['name'], subScene['summary'], subScene['name']))
                print(f'Dialogues generated: {len(json.loads(rawDialogues))}')
                recordedDialoges = []
                for dialogue in parsedDialoges:
                    dialogue['id'] = str(uuid4().hex)
                    collection.add(documents=[json.dumps({'name':dialogue['name'], 'text':dialogue['text']})], metadatas=[{"source": "my_source"}], ids=[dialogue['id']])
                    recordedDialoges.append(dialogue)
                
                dialoguesSoFar += recordedDialoges
            except:
                pass

    for dialogue in dialoguesSoFar:
        print(f"{dialogue['name']}: {dialogue['text']}")


Overall Plot


{'setting': 'A dystopian universe where the government is oppressive and lawless',
 'characters': [{'name': 'John',
   'background': 'A rebel who fights against the oppressive government'},
  {'name': 'Jane', 'background': 'A leader of the rebellion'},
  {'name': 'The Government', 'background': 'An oppressive and lawless force'}],
 'conflict': 'John and Jane must fight to overthrow the oppressive government and restore freedom to the people.',
 'plotTwist': 'The government has a secret weapon that could destroy the rebellion and the people.',
 'resolution': 'John and Jane are able to find and destroy the secret weapon, allowing the rebellion to succeed and the people to be free.'}

Overall Scenes


[{'name': 'John and Jane meet',
  'summary': 'John and Jane meet for the first time and discuss the oppressive government and their plans to overthrow it. They realize they must work together to succeed.'},
 {'name': 'The Rebellion Grows',
  'summary': 'John and Jane begin to recruit more people to join their rebellion and form a plan to overthrow the oppressive government.'},
 {'name': "The Government's Secret Weapon",
  'summary': "John and Jane discover the government's secret weapon and realize they must find and destroy it before the government can use it against them."},
 {'name': 'The Final Battle',
  'summary': 'John and Jane lead the rebellion in a final battle against the government. They are able to find and destroy the secret weapon, allowing the rebellion to succeed and the people to be free.'}]

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
Exiting: Cleaning up .chroma directory
John and Jane meet: John and Jane meet for the first time and discuss the oppressive government and their plans to overthrow it. They realize they must work together to succeed.

Sub Plot


{'setting': "John and Jane's first meeting",
 'characters': [{'name': 'John',
   'background': 'A rebel seeking to overthrow the oppressive government'},
  {'name': 'Jane',
   'background': 'A rebel seeking to overthrow the oppressive government'}],
 'conflict': 'John and Jane must work together to overthrow the oppressive government',
 'plotTwist': 'The oppressive government has a powerful ally that is willing to do whatever it takes to keep them in power',
 'resolution': 'John and Jane join forces with other rebels and successfully overthrow the oppressive government'}

Sub Scenes


[{'name': "John and Jane's First Meeting",
  'summary': 'John and Jane meet for the first time in a secluded location. Jane is a bit hesitant at first but John is determined to get her to join his cause. They discuss their shared goals of overthrowing the oppressive government and decide to work together to make it happen.'},
 {'name': 'Planning the Rebellion',
  'summary': 'John and Jane spend the next few days planning their rebellion. They meet with other rebels, gather resources, and come up with a plan of attack. They soon realize that the oppressive government has a powerful ally that is willing to do whatever it takes to keep them in power.'},
 {'name': 'The Final Battle',
  'summary': 'John and Jane lead the rebels in a final battle against the oppressive government. With the help of their allies, they are able to overpower the oppressive government and successfully overthrow them. John and Jane emerge victorious and the people of the country are finally free.'}]

passed dialouges []
Dialogues generated: 4
parsed good
similarDialogues [['{"name": "John", "text": "Hi, I\'m John. I\'m here to talk to you about overthrowing the oppressive government."}', '{"name": "John", "text": "I understand your hesitation, but I think we can make a difference if we work together. We can make a real change in this country."}']]
passed dialouges [['{"name": "John", "text": "Hi, I\'m John. I\'m here to talk to you about overthrowing the oppressive government."}', '{"name": "John", "text": "I understand your hesitation, but I think we can make a difference if we work together. We can make a real change in this country."}'], {'name': 'John', 'text': "Hi, I'm John. I'm here to talk to you about overthrowing the oppressive government.", 'id': 'c47ed62bcb55416a8978d8028972cc90'}, {'name': 'Jane', 'text': "I'm Jane. I'm not sure if I'm ready to join your cause, but I'm willing to listen.", 'id': 'a4c8e4b0867a47099fec4227fea87245'}, {'name': 'John', 'text': 'I understand